In [ ]:
%AddDeps org.elasticsearch elasticsearch-spark-20_2.11 6.8.9
%AddDeps com.datastax.spark spark-cassandra-connector_2.11 2.4.3
%AddDeps org.postgresql postgresql 42.2.12

# Elasticsearch

In [ ]:
val ES_USERNAME: String = "***"
val ES_PASSWORD: String = "***"
val ELASTIC_HOST = "10.0.0.5:9200"

In [ ]:
import org.elasticsearch.spark.sql

In [ ]:
val esOptions =
    Map(
      "es.nodes" -> ELASTIC_HOST,
      "es.batch.write.refresh" -> "false",
      "es.net.http.auth.user" -> ES_USERNAME,
      "es.net.http.auth.pass" -> ES_PASSWORD,
      "es.nodes.wan.only" -> "true"
    )

In [ ]:
var shops = spark
                .read
                .format("org.elasticsearch.spark.sql")
                .options(esOptions)
                .load("visits")
                .toDF

# HDFS

In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

In [ ]:
val input = spark.read
                .option("header",true)
                .json("/labs/laba03/weblogs.json").toDF
                .select('uid, explode(col("visits")))
                .select('uid, col("col.*"))
                .toDF

In [ ]:
val webLogs = input.na.drop(List("uid"))

# Cassandra

In [ ]:
val CASSANDRA_HOST = "10.0.0.5"
val CASSANDRA_PORT = "9042"

In [15]:
spark.conf.set("spark.cassandra.connection.host", CASSANDRA_HOST)
spark.conf.set("spark.cassandra.connection.port", CASSANDRA_PORT)
spark.conf.set("spark.cassandra.output.consistency.level", "ANY")
spark.conf.set("spark.cassandra.input.consistency.level", "ONE")

In [ ]:
val cOpts = Map("table" -> "clients", "keyspace" -> "labdata")

In [ ]:
val clients = spark
                .read
                .format("org.apache.spark.sql.cassandra")
                .options(cOpts)
                .load
                .toDF

# PostgreSQL

In [ ]:
val POSTGRE_USER = "***"
val POSTGRE_URL = s"jdbc:postgresql://10.0.0.5:5432/labdata?user=$POSTGRE_USER&password=$ES_PASSWORD"

In [54]:
import org.postgresql.Driver

In [ ]:
val pgOptions = Map(
        "url" -> POSTGRE_URL,
        "dbtable" -> "domain_cats",
        "user" -> POSTGRE_USER,
        "password" -> ES_PASSWORD,
        "driver" -> "org.postgresql.Driver"
        )

In [ ]:
val webCats = spark
    .read
    .format("jdbc")
    .options(pgOptions)
    .load
    .toDF

# Main part

Используя **psql**, создайте в вашей базе данных name_surname таблицу clients со следующими колонками:

*uid, gender, age_cat, shop_cat1, ... , shop_catN, web_cat1, ... , web_catN*

где:

* **uid** (primary key) – uid пользователя.
* **gender** – пол пользователя: M, F.
* **age_cat** – категория возраста, одна из пяти: 18-24, 25-34, 35-44, 45-54, >=55.
* **shop_cat**, **web_cat** – категории товаров и категории веб-сайтов.

##  0. Clients

In [ ]:
val new_col = when(clients("age").between(18, 24), "18-24")
                .when(clients("age").between(25, 34), "25-34")
                .when(clients("age").between(35, 44), "35-44")
                .when(clients("age").between(45, 54), "45-54")
                .when(clients("age") >= 55, ">=55")
val cat_clients_age = clients.withColumn("age_cat", new_col)
val cat_clients = cat_clients_age.drop("age")

## 1. Websites & categories

In [64]:
import org.apache.spark.sql.functions.udf
import scala.util.Try
import java.net.URL
import java.net.URLDecoder.decode

In [ ]:
val decode_url = udf { (url: String) => Try(new URL(decode(url, "UTF-8")).getHost).toOption}

In [ ]:
val filtered_logs = webLogs
            .filter('url.startsWith("http"))
            .withColumn("url", decode_url(col("url")))
            .withColumn("url", regexp_replace('url, "^www.", ""))
            .dropDuplicates

In [ ]:
val websitesWithCats = filtered_logs
                .join(webCats, filtered_logs("url") === webCats("domain"))
                .groupBy("uid", "category").count
                .withColumn("category", concat(lit("web_"), col("category")))

In [ ]:
val usersToCats = websitesWithCats
    .groupBy("uid")
    .pivot("category")
    .sum("count")
    .na.fill(0)

## 2. Shops & categories

In [ ]:
val shopsWithCats = shops
            .select('uid, 'category)
            .withColumn("category", lower(col("category")))
            .withColumn("category", regexp_replace('category, "[ -]", "_"))
            .groupBy("uid", "category").count
            .withColumn("category", concat(lit("shop_"), col("category")))

In [ ]:
val usersToShops = shopsWithCats
    .groupBy("uid")
    .pivot("category")
    .sum("count")
    .na.fill(0)

## 3. Join Part

In [ ]:
val right = usersToShops.withColumnRenamed("uid", "right_uid")

In [ ]:
val tmp = cat_clients
            .join(right, right("right_uid") === cat_clients("uid"), "left")
            .drop("right_uid")
            .na.fill(0)

In [ ]:
val right1 = usersToCats.withColumnRenamed("uid", "right_uid")

In [ ]:
val res = tmp
            .join(right1, right1("right_uid") === tmp("uid"), "left")
            .drop("right_uid")
            .na.fill(0)

### Saving result in PostgreSQL

In [ ]:
val writeOptions = Map(
        "url" -> "jdbc:postgresql://10.0.0.5:5432/***",
        "dbtable" -> "clients",
        "user" -> POSTGRE_USER,
        "password" -> ES_PASSWORD,
        "driver" -> "org.postgresql.Driver")

In [ ]:
res
    .write
    .format("jdbc")
    .options(writeOptions)
    .mode("overwrite")
    .save